# Ceneo Scraper

## Components of single opinion

|Component|Selector|Variable|
|---------|--------|--------|
|opinion ID|["data-entry-id"]|opinion_id|
|opinion’s author|span.user-post__author-name|author|
|author’s recommendation|span.user-post__author-recomendation > em|recommendation|
|score expressed in number of stars|span.user-post__score-count|score|
|opinion’s content|div.user-post__text|content|
|list of product advantages|div.review-feature__title--positives ~ div.review-feature__item|pros|
|list of product disadvantages|div.review-feature__title--negatives ~ div.review-feature__item|cons|
|how many users think that opinion was helpful|button.vote-yes > span|helpful|
|how many users think that opinion was unhelpful|button.vote-no > span|unhelpful|
|publishing date|span.user-post__published > time:nth-child(1)["datetime"]|publish_date|
|purchase date|span.user-post__published > time:nth-child(2)["datetime"]|purchase_date|

## Structure of single opinion

In [1]:
selectors = {
    "opinion_id": [None,"data-entry-id"],
    "author": ["span.user-post__author-name"],
    "recommendation": ["span.user-post__author-recomendation > em"],
    "score": ["span.user-post__score-count"],
    "content": ["div.user-post__text"],
    "pros": ["div.review-feature__title--positives ~ div.review-feature__item", None, True],
    "cons": ["div.review-feature__title--negatives ~ div.review-feature__item", None, True],
    "helpful": ["button.vote-yes > span"],
    "unhelpful": ["button.vote-no > span"],
    "publish_date": ["span.user-post__published > time:nth-child(1)","datetime"],
    "purchase_date": ["span.user-post__published > time:nth-child(2)","datetime"],
}

## Loading libraries

In [2]:
import requests
from bs4 import BeautifulSoup

## Function to extract data from HTML code

In [3]:
def extract(ancestor, selector=None, attribute=None, return_list=False):
    if return_list:
        if attribute:
            return [tag[attribute] for tag in ancestor.select(selector)]
        return [tag.get_text().strip() for tag in ancestor.select(selector)]
    if selector:
        if attribute:
            try:
                return ancestor.select_one(selector)[attribute]
            except TypeError:
                return None
        try:   
            return ancestor.select_one(selector).get_text().strip()
        except AttributeError:
            return None
    if attribute:
        return ancestor[attribute]
    return ancestor.get_text().strip()

## Sending request to Ceneo.pl server

In [4]:
product_id = "13029872"
url = f"https://www.ceneo.pl/{product_id}#tab=reviews"
response = requests.get(url)

## Extracting opinions from HTML code

In [5]:
page_dom = BeautifulSoup(response.text, "html.parser")
opinions = page_dom.select("div.js_product-review")
opinion = page_dom.select_one("div.js_product-review")


## Extracting components for single opinion

In [6]:
single_opinion = {
    key: extract(opinion, *value) 
        for key, value in selectors.items()
}